# Maitrisez le RAG et créez vos propres ChatGPT



## 1. Les bases de LangChain

### 1.1 Installation

Installons les dépendances et préparons notre clé OpenAI

In [1]:
!pip install langchain
!pip install langchain-openai

  Using cached langchain-0.3.19-py3-none-any.whl (1.0 MB)
  Using cached langchain_core-0.3.40-py3-none-any.whl (414 kB)
  Using cached langchain_text_splitters-0.3.6-py3-none-any.whl (31 kB)
  Using cached langsmith-0.3.11-py3-none-any.whl (335 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl (431 kB)
  Using cached aiohttp-3.11.13-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)
  Using cached tenacity-9.0.0-py3-none-any.whl (28 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl (7.6 kB)
  Using cached frozenlist-1.5.0-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (274 kB)
  Using cached multidict-6.1.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (129 kB)
  Using cached propcache-0.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (231 kB)
  Using cached yarl-1.18.3-cp311-cp311-manyl

Pour suivre ce tuto, vous devrez [**générer une clé d'API OpenAI**](https://platform.openai.com/api-keys)

In [10]:
# Déclarez votre clé d'API OPENAI
import os

os.environ['OPENAI_API_KEY'] = "API_KEY" 

### 1.2 Création de votre premier modèle

Vous êtes maintenant prêt pour créer et utiliser votre premier modèle LangChain. Ici nous allons créer un modèle qui se connectera à OpenAI. 

In [ ]:
from pprint import pprint
from langchain_openai import ChatOpenAI

# Création du model LangChain OpenAI chat
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0.9,
    max_tokens=500
)

# Test du model avec une première question
ai_response = llm.invoke("Qu'est-ce qu'une IA générative ?")
print(ai_response.content)

# metadata de la requête
print('-' * 40)
pprint(ai_response.response_metadata)

### 1.3 Création d'un prompt

Vous allez pour commençer à tirer profit de la puissance de LangChain.   
Imaginez que nous voulons créer une fonction qui prend en entrée une couleur et qui renvoi une liste d'animaux qui sont de cette couleur.   
Vous allez devoir créer un prompt plus élaboré :

ChatPromptTemplate : Permet de structurer un prompt pour un chatbot.

SystemMessagePromptTemplate : Définit les instructions générales pour l'IA.

HumanMessagePromptTemplate : Définit l'entrée utilisateur.

In [6]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Création d'un Prompt Template
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("Tu es un assistant qui renvoie une liste d'animaux qui ont une couleur donnée. Retourne les résultats dans une liste json."),
    HumanMessagePromptTemplate.from_template("Quels animaux ont la couleur {color}.")
])


def get_animals(color):
    # Formatage du prompt avec le paramètre d'entré 
    prompt_messages = chat_prompt.format_prompt(color=color).to_messages()

    # Appel LLM avec les messages de prompt
    return llm.invoke(prompt_messages).content
    

In [ ]:
get_animals('rouge')

### 1.4 Utilisation d'un parseur

La réponse est valide mais on aimerait pouvoir l'exploiter. Il faut donc la parser pour l'avoir dans un objet python.

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

output_parser = JsonOutputParser()

def get_animals(color):
    # Formatage du prompt avec le paramètre d'entré 
    prompt_messages = chat_prompt.format_prompt(color=color).to_messages()

    # Appel LLM avec les messages de prompt
    result_string = llm.invoke(prompt_messages).content

    # Parsing du résultat
    return output_parser.invoke(result_string)

In [ ]:
get_animals('rouge')

## 2. Composition de chaines

En ajoutant des étapes supplémentaire à notre programme LangChain, celui-ci peut rapidement devenir illisible et complexe.   
On va donc utiliser les chaines LangChain pour faciliter la construction et la lecture de notre programme.   
Reprenons d'abord les différents bloque LangChain du programme que nous avons vu précédemment :

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.9,
    max_tokens=500
)

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("Tu es un assistant qui renvoie une liste d'animaux qui ont une couleur donnée. Retourne les résultats dans une liste json."),
    HumanMessagePromptTemplate.from_template("Quels animaux ont la couleur {color}.")
])

output_parser = JsonOutputParser()

Initialement, la fonction ressemblait à ça :

In [ ]:
def get_animals(color):
    # Formatage du prompt avec le paramètre d'entré 
    prompt_messages = chat_prompt.format_prompt(color=color).to_messages()

    # Appel LLM avec les messages de prompt
    result_string = llm.invoke(prompt_messages).content

    # Parsing du résultat
    return output_parser.invoke(result_string)

Avec les chaines, nous pouvons l'écrire de la manière suivante :

In [ ]:
def get_animals(color):
    chain = chat_prompt | llm | output_parser
    return chain.invoke(color)

In [ ]:
get_animals('blanc')

## 3. Création d'un RAG

Installons d'abord quelques dépendances nécessaires

In [ ]:
!pip install pymupdf
!pip install chromadb

Puis nous devons préparer notre base vectorisé.

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

PDF_FILE_PATH = "rapport-commision-IA-france-mars-2024.pdf"

# Chargement et parsing du PDF
loader = PyMuPDFLoader(PDF_FILE_PATH)
documents = loader.load()

# Découpage en chunk
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=250)
doc_chunks = text_splitter.split_documents(documents)

# Construction du vector store
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents=doc_chunks,
                                    embedding=embeddings)

# Document retriever
doc_retriever = vectorstore.as_retriever(top_k=10)

Testons notre chercheur de document.

In [ ]:
# TODO : To fix
docs = doc_retriever.invoke("Qu'en est-il de la souveraineté ?")
for doc in docs:
    print(f"Page {doc.metadata['page']} : {doc.page_content[:70]}...")

Ensuite, on prépare le prompt, une fonction pour formater les documents et un parser de sortie.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# Ce prompt aura 2 variables d'entrée : context et question
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template_file("prompt_system.txt", input_variables=[]),
    SystemMessagePromptTemplate.from_template_file("prompt_context.txt", input_variables=["context"]),
    HumanMessagePromptTemplate.from_template("{question}")
])

# Fonction pour formater les documents récupérés
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Parser de sortie
output_parser = StrOutputParser()

On peut maintenant construire la chaine finale !

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain = (
    {"context": doc_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

Et voila !

In [ ]:
print(chain.invoke("Comment financer l’émergence de l’écosystème d’IA en France ?"))

## 4. Surveiller vos interactions avec votre LLM grâce à LangSmith

Installons les LangSmith et préparons notre clé OpenAI

In [ ]:
!pip install langsmith

Pour suivre ce tuto, vous devrez [**générer une clé d'API LangSmith**](https://smith.langchain.com/settings)

In [ ]:
# Déclarez votre clé d'API LangSmith
import os

os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_API_KEY'] = "<YOUR-LANGSMITH-API-KEY>"

Utilisons de nouveau notre chaine pour que LangSmith puisse en capturer les traces :

In [ ]:
print(chain.invoke("Comment financer l’émergence de l’écosystème d’IA en France ?"))

Maintenant, nous pouvons consulter les traces de notre chaine sur LangSmith